In [1]:
import os
import openai
import sys
sys.path.append('../..')

import panel as pn  # GUI
pn.extension()

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

In [4]:
def load_db(file, chain_type, k):
    # load documents
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    # define embedding
    embeddings = OpenAIEmbeddings()
    # create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    # define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chatbot chain. Memory is managed externally.
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name=llm_name, temperature=0), 
        chain_type=chain_type, 
        retriever=retriever, 
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa 


In [5]:
import panel as pn
import param

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])
    
    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.loaded_file = "docs/papers/paper_4.pdf"
        self.qa = load_db(self.loaded_file,"stuff", 4)
    
    def call_load_db(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            file_input.save("temp.pdf")  # local copy
            self.loaded_file = file_input.filename
            button_load.button_style="outline"
            self.qa = load_db("temp.pdf", "stuff", 4)
            button_load.button_style="solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, style={'background-color': '#F6F6F6'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return 
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history') 
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    def clr_history(self,count=0):
        self.chat_history = []
        return 


In [6]:
cb = cbfs()

file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput( placeholder='Enter text here…')

bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp) 

jpg_pane = pn.pane.Image( 'images/photo_1.png')

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4=pn.Column(
    pn.Row( file_input, button_load, bound_button_load),
    pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" )),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400))
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3),('Configure', tab4))
)
dashboard

C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\pydantic\_migration.py:281: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


BokehModel(combine_events=True, render_bundle={'docs_json': {'88d7c6e1-664f-4a7c-98e2-c98d72afca8b': {'version…

2023-11-02 11:50:50,488 ERROR: panel.reactive - Callback failed for object named "" changing property {'value': 'wha'} 
Traceback (most recent call last):
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\panel\reactive.py", line 385, in _process_events
    self.param.update(**self_events)
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\param\parameterized.py", line 2282, in update
    restore = self_._update(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\param\parameterized.py", line 2322, in _update
    self_._batch_call_watchers()
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\param\parameterized.py", line 2506, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\param\parameterized.py", line 2468, in _execute_watcher
    watcher.fn(*args, **kwargs)
 

ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [-0.0042954... -0.007739575508156901]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.4/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [-0.0042954... -0.007739575508156901]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.4/v/missing

2023-11-02 11:51:00,065 ERROR: panel.reactive - Callback failed for object named "" changing property {'value': 'what is the document a'} 
Traceback (most recent call last):
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\panel\reactive.py", line 385, in _process_events
    self.param.update(**self_events)
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\param\parameterized.py", line 2282, in update
    restore = self_._update(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\param\parameterized.py", line 2322, in _update
    self_._batch_call_watchers()
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\param\parameterized.py", line 2506, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\param\parameterized.py", line 2468, in _execute_watcher
    watcher.fn

ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [-0.0039039... -0.042565012813003136]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.4/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [-0.0039039... -0.042565012813003136]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.4/v/missing

2023-11-02 11:51:23,797 ERROR: panel.reactive - Callback failed for object named "" changing property {'value': 'what is the document ab'} 
Traceback (most recent call last):
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\panel\reactive.py", line 385, in _process_events
    self.param.update(**self_events)
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\param\parameterized.py", line 2282, in update
    restore = self_._update(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\param\parameterized.py", line 2322, in _update
    self_._batch_call_watchers()
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\param\parameterized.py", line 2506, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\param\parameterized.py", line 2468, in _execute_watcher
    watcher.f

ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [-0.0196004..., -0.03111328665906562]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.4/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [-0.0196004..., -0.03111328665906562]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.4/v/missing

2023-11-02 11:52:14,719 ERROR: panel.reactive - Callback failed for object named "" changing property {'value': 'who is the author of the document'} 
Traceback (most recent call last):
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\panel\reactive.py", line 385, in _process_events
    self.param.update(**self_events)
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\param\parameterized.py", line 2282, in update
    restore = self_._update(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\param\parameterized.py", line 2322, in _update
    self_._batch_call_watchers()
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\param\parameterized.py", line 2506, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "C:\Users\alanm\AppData\Roaming\Python\Python311\site-packages\param\parameterized.py", line 2468, in _execute_watcher
   

ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [-0.0022867...4, -0.0332748571286292]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.4/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [-0.0022867...4, -0.0332748571286292]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.4/v/missing